In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip '/content/drive/MyDrive/Used Vehicles.csv' -d file_destination

Archive:  /content/drive/MyDrive/Used Vehicles.csv
replace file_destination/vehicles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: file_destination/vehicles.csv  


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/file_destination/vehicles.csv')
df.head(5)

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,0,7240372487,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,35990,2010.0,chevrolet,corvette grand sport,good,8 cylinders,gas,32742.0,clean,other,1G1YU3DW1A5106980,rwd,NaN,other,NaN,https://images.craigslist.org/00N0N_ipkbHVZYf4...,Carvana is the safer way to buy a car During t...,al,32.590000,-85.480000,2020-12-02T08:11:30-0600
1,1,7240309422,https://auburn.craigslist.org/cto/d/auburn-201...,auburn,https://auburn.craigslist.org,7500,2014.0,hyundai,sonata,excellent,4 cylinders,gas,93600.0,clean,automatic,5NPEC4AB0EH813529,fwd,NaN,sedan,NaN,https://images.craigslist.org/00s0s_gBHYmJ5o7y...,I'll move to another city and try to sell my c...,al,32.547500,-85.468200,2020-12-02T02:11:50-0600
2,2,7240224296,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,4900,2006.0,bmw,x3 3.0i,good,6 cylinders,gas,87046.0,clean,automatic,NaN,NaN,NaN,SUV,blue,https://images.craigslist.org/00B0B_5zgEGWPOrt...,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al,32.616807,-85.464149,2020-12-01T19:50:41-0600
3,3,7240103965,https://auburn.craigslist.org/cto/d/lanett-tru...,auburn,https://auburn.craigslist.org,2000,1974.0,chevrolet,c-10,good,4 cylinders,gas,190000.0,clean,automatic,NaN,rwd,full-size,pickup,blue,https://images.craigslist.org/00M0M_6o7KcDpArw...,1974 chev. truck (LONG BED) NEW starter front ...,al,32.861600,-85.216100,2020-12-01T15:54:45-0600
4,4,7239983776,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,19500,2005.0,ford,f350 lariat,excellent,8 cylinders,diesel,116000.0,lien,automatic,NaN,4wd,full-size,pickup,blue,https://images.craigslist.org/00p0p_b95l1EgUfl...,2005 Ford F350 Lariat (Bullet Proofed). This t...,al,32.547500,-85.468200,2020-12-01T12:53:56-0600


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458213 entries, 0 to 458212
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    458213 non-null  int64  
 1   id            458213 non-null  int64  
 2   url           458213 non-null  object 
 3   region        458213 non-null  object 
 4   region_url    458213 non-null  object 
 5   price         458213 non-null  int64  
 6   year          457163 non-null  float64
 7   manufacturer  439993 non-null  object 
 8   model         453367 non-null  object 
 9   condition     265273 non-null  object 
 10  cylinders     287073 non-null  object 
 11  fuel          454976 non-null  object 
 12  odometer      402910 non-null  float64
 13  title_status  455636 non-null  object 
 14  transmission  455771 non-null  object 
 15  VIN           270664 non-null  object 
 16  drive         324025 non-null  object 
 17  size          136865 non-null  object 
 18  type

In [6]:
df.describe(include='all')

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
count,458213.000000,4.582130e+05,458213,458213,458213,4.582130e+05,457163.000000,439993,453367,265273,287073,454976,4.029100e+05,455636,455771,270664,324025,136865,345475,317370,458185,458143,458213,450765.000000,450765.000000,458185
unique,NaN,NaN,458213,405,413,NaN,NaN,43,31520,6,8,5,NaN,6,3,133620,3,4,13,12,285404,384347,51,NaN,NaN,423805
top,NaN,NaN,https://boulder.craigslist.org/ctd/d/boulder-2...,columbus,https://bend.craigslist.org,NaN,NaN,ford,f-150,good,6 cylinders,gas,NaN,clean,automatic,1C4HJWDG6HL702229,4wd,full-size,sedan,white,https://images.craigslist.org/00c0c_1gCUGosl6w...,"👉 BAD Credit, NO Credit - OKAY! We FINANCE ANY...",ca,NaN,NaN,2020-12-02T07:50:07-0600
freq,NaN,NaN,1,3818,2989,NaN,NaN,79666,8370,115685,105677,399399,NaN,439626,396322,194,139693,69736,92364,82786,194,247,51856,NaN,NaN,7
mean,229106.000000,7.235233e+09,NaN,NaN,NaN,4.042093e+04,2010.746067,NaN,NaN,NaN,NaN,NaN,1.016698e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.531925,-94.375824,NaN
std,132274.843786,4.594362e+06,NaN,NaN,NaN,8.194599e+06,8.868136,NaN,NaN,NaN,NaN,NaN,3.228623e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.857378,18.076225,NaN
min,0.000000,7.208550e+09,NaN,NaN,NaN,0.000000e+00,1900.000000,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.607549,-164.091797,NaN
25%,114553.000000,7.231953e+09,NaN,NaN,NaN,4.900000e+03,2008.000000,NaN,NaN,NaN,NaN,NaN,4.087700e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.600000,-110.890427,NaN
50%,229106.000000,7.236409e+09,NaN,NaN,NaN,1.099500e+04,2013.000000,NaN,NaN,NaN,NaN,NaN,8.764100e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.244500,-88.314889,NaN
75%,343659.000000,7.239321e+09,NaN,NaN,NaN,2.149500e+04,2016.000000,NaN,NaN,NaN,NaN,NaN,1.340000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.484503,-81.015022,NaN


In [7]:
#following the columns Foba maintained
df = df[['region', 'year', 'manufacturer', 'model', 'cylinders', 'fuel', 'odometer', 'description', 'title_status', 'transmission', 'drive', 'type', 'paint_color', 'state', 'price']]

In [8]:
df.shape

(458213, 15)

In [9]:
df.head()

,region,year,manufacturer,model,cylinders,fuel,odometer,description,title_status,transmission,drive,type,paint_color,state,price
0,auburn,2010.0,chevrolet,corvette grand sport,8 cylinders,gas,32742.0,Carvana is the safer way to buy a car During t...,clean,other,rwd,other,NaN,al,35990
1,auburn,2014.0,hyundai,sonata,4 cylinders,gas,93600.0,I'll move to another city and try to sell my c...,clean,automatic,fwd,sedan,NaN,al,7500
2,auburn,2006.0,bmw,x3 3.0i,6 cylinders,gas,87046.0,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,clean,automatic,NaN,SUV,blue,al,4900
3,auburn,1974.0,chevrolet,c-10,4 cylinders,gas,190000.0,1974 chev. truck (LONG BED) NEW starter front ...,clean,automatic,rwd,pickup,blue,al,2000
4,auburn,2005.0,ford,f350 lariat,8 cylinders,diesel,116000.0,2005 Ford F350 Lariat (Bullet Proofed). This t...,lien,automatic,4wd,pickup,blue,al,19500


In [10]:
df.isnull().sum()

region               0
year              1050
manufacturer     18220
model             4846
cylinders       171140
fuel              3237
odometer         55303
description         70
title_status      2577
transmission      2442
drive           134188
type            112738
paint_color     140843
state                0
price                0
dtype: int64

# **Filling up columns with the highest numbers of missing values:'drive', 'type', 'paint_color' and 'manufacturer' from the description column**:

**Observation**
There are 3 unique drives, 13 unique types, 12 paint colors, 3 unique transmissions and 8 unique cylinders.

New columns will be created for description, drive, type, paint_color and cylinder

**DESCRIPTION**

In [11]:
#Converting strings within the description column to lower case

df['new_description'] = df['description'].astype(str).apply(lambda x: x.lower())

df['new_description']

0         carvana is the safer way to buy a car during t...
1         i'll move to another city and try to sell my c...
2         clean 2006 bmw x3 3.0i.  beautiful and rare bl...
3         1974 chev. truck (long bed) new starter front ...
4         2005 ford f350 lariat (bullet proofed). this t...
                                ...                        
458208    2014 cadillac srx all wheel drive luxury colle...
458209    2018 hyundai tucson se 2.0 i4 awd.  carfax ver...
458210    53,000 miles. cosmetically, in near mint condi...
458211    price reduction  turns out the engine is toast...
458212    *** shipping available directly to you. call f...
Name: new_description, Length: 458213, dtype: object

**DRIVE**

In [12]:
drive = df.groupby('drive')['drive'].count()
drive

drive
4wd    139693
fwd    122602
rwd     61730
Name: drive, dtype: int64

In [13]:
#Creating a new column for drive by searching for occurrences of the 3 drive types within the description column


df['newdrive'] = df.new_description.str.findall('(4wd|fwd|rwd)')

df['newdrive'] = df['newdrive'].apply(', '.join)

#if drive is NAN, replace with the value in 'new_drive'
df.loc[df['drive'].isnull(), 'drive'] =  df['newdrive']


In [14]:
#df['newdrive']

In [15]:
df[['drive','newdrive']].sample(50)

,drive,newdrive
400960,,
361275,fwd,
108371,,
332327,4wd,
328769,4wd,"4wd, 4wd, 4wd"
212965,fwd,fwd
377970,rwd,
411405,,
20409,rwd,"rwd, 4wd, 4wd, 4wd"
117545,"4wd, 4wd, 4wd","4wd, 4wd, 4wd"


In [16]:
df[df['drive'] == ''].sum

<bound method DataFrame.sum of              region  ...  newdrive
2            auburn  ...          
12           auburn  ...          
13           auburn  ...          
18           auburn  ...          
20           auburn  ...          
...             ...  ...       ...
458194  northern WI  ...          
458197    sheboygan  ...          
458205       wausau  ...          
458207       wausau  ...          
458212      wyoming  ...          

[105984 rows x 17 columns]>

**Observation**

A TOTAL OF 105984 ROWS ARE STILL MISSING. 

HOWEVER, THAT IMPLIES THERE ARE AN ADDITIONAL 27,204 VALID ROWS ADDED TO THE     'drive'  COLUMN FROM THE 134,188 ROWS THAT WERE PREVIOUSLY NAN

**TYPE**

In [17]:
type = df.groupby('type')['type'].count()
type

type
SUV            79882
bus              717
convertible     6864
coupe          19106
hatchback      16388
mini-van        6263
offroad          647
other          24160
pickup         43343
sedan          92364
truck          35694
van             9574
wagon          10473
Name: type, dtype: int64

In [18]:
#excluding 'other'
df['new_type'] = df.new_description.str.findall('(suv|bus|convertible|coupe|hatchback|mini-van|offroad|pickup|sedan|truck|van|wagon)').apply(', '.join)

df.loc[df['type'].isnull(), 'type'] =  df['new_type']

In [19]:
df[['type','new_type']]

,type,new_type
0,other,"van, van, van, van, van, van, van, van, van, v..."
1,sedan,
2,SUV,van
3,pickup,truck
4,pickup,"truck, truck, truck, truck, truck, truck, truck"
...,...,...
458208,SUV,bus
458209,SUV,bus
458210,coupe,coupe
458211,SUV,truck


In [20]:
df[df['type'] == ''].sum

<bound method DataFrame.sum of              region    year  ... newdrive new_type
13           auburn  2010.0  ...                  
20           auburn  2003.0  ...                  
44           auburn  1986.0  ...                  
62           auburn  2007.0  ...                  
102      birmingham  1961.0  ...                  
...             ...     ...  ...      ...      ...
458179    milwaukee  2002.0  ...                  
458186    milwaukee  2013.0  ...                  
458187    milwaukee  2014.0  ...                  
458194  northern WI  2013.0  ...                  
458198    sheboygan  1993.0  ...      4wd         

[53293 rows x 18 columns]>

**Observation** 

A TOTAL OF 53,293 ROWS ARE STILL EMPTY 

HOWEVER, A TOTAL OF 59,445 ROWS HAVE BEEN FILLED. ALTHOUGH SOME OF THE ROWS MIGHT STILL BE DROPPED OR CLEANED SINCE THEY HAVE MULTIPLE ENTRIES.

**PAINT_COLOR**

In [21]:
paint_color = df.groupby('paint_color')['paint_color'].count()
paint_color

paint_color
black     64145
blue      32746
brown      6028
custom     8843
green      7631
grey      30455
orange     1970
purple      768
red       33274
silver    46722
white     82786
yellow     2002
Name: paint_color, dtype: int64

In [22]:
df['paint_color']

0            NaN
1            NaN
2           blue
3           blue
4           blue
           ...  
458208    silver
458209     white
458210     black
458211     white
458212       NaN
Name: paint_color, Length: 458213, dtype: object

In [23]:
df['new_paint_color'] = df.new_description.str.findall('(black|blue|brown|custom|green|grey|orange|purple|red|silver|white|yellow|gray)').apply(', '.join)

df.loc[df['paint_color'].isnull(), 'paint_color'] =  df['new_paint_color']

In [24]:
df[['paint_color','new_paint_color']]

,paint_color,new_paint_color
0,"custom, custom, red, red, gray, black, red, si...","custom, custom, red, red, gray, black, red, si..."
1,blue,blue
2,blue,blue
3,blue,
4,blue,
...,...,...
458208,silver,
458209,white,blue
458210,black,
458211,white,"red, red, red, red, blue, white, black, red, red"


In [25]:
df[df['paint_color'] == ''].sum

<bound method DataFrame.sum of              region  ...  new_paint_color
13           auburn  ...                 
20           auburn  ...                 
44           auburn  ...                 
88       birmingham  ...                 
102      birmingham  ...                 
...             ...  ...              ...
458177    milwaukee  ...                 
458182    milwaukee  ...                 
458186    milwaukee  ...                 
458194  northern WI  ...                 
458212      wyoming  ...                 

[39029 rows x 19 columns]>

**Observation** 

A TOTAL OF 39,572 ROWS ARE STILL EMPTY 

HOWEVER, A TOTAL OF 101,271 ROWS HAVE BEEN FILLED. ALTHOUGH SOME OF THE ROWS STILL NEEDS TO BE DROPPED OR CLEANED SINCE THERE ARE MULTIPLE ENTRIES.

**MANUFACTURER**

In [26]:
df['new_manufacturer'] = df.new_description.str.findall('(volvo|volkswagen|toyota|tesla|subaru|saturn|rover|ram|porsche|pontiac|nissan|morgan|mitsubishi|mini|mercury|mercedes-benz|mazda|lincoln|lexus|land rover|kia|jeep|jaguar|infiniti|hyundai|honda|hennessey|harley-davidson|gmc|ford|fiat|ferrari|dodge|datsun|chrysler|chevrolet|cadillac|buick|bmw|audi|aston-martin|alfa-romeo|acura)').apply(', '.join)

df.loc[df['manufacturer'].isnull(), 'manufacturer'] =  df['new_manufacturer']


In [27]:
df[['manufacturer','new_manufacturer']]

,manufacturer,new_manufacturer
0,chevrolet,"chevrolet, ram, chevrolet, chevrolet, chevrole..."
1,hyundai,
2,bmw,bmw
3,chevrolet,
4,ford,ford
...,...,...
458208,cadillac,cadillac
458209,hyundai,hyundai
458210,cadillac,
458211,jeep,"jeep, jeep, audi, jeep, jeep, jeep"


In [28]:
df[df['manufacturer'] == ''].sum

<bound method DataFrame.sum of             region  ...  new_manufacturer
21          auburn  ...                  
227     birmingham  ...                  
238     birmingham  ...                  
319     birmingham  ...                  
323     birmingham  ...                  
...            ...  ...               ...
458107     madison  ...                  
458115     madison  ...                  
458118     madison  ...                  
458162   milwaukee  ...                  
458163   milwaukee  ...                  

[8572 rows x 20 columns]>

A TOTAL OF 8,572 ROWS STILL HAVE 'manufacturer' AS EMPTY

HOWEVER, A TOTAL OF 9,650 ROWS HAVE BEEN FILLED. 

# **FURTHER CLEANING**

In [46]:
#df = df.drop(columns=['new_description','newdrive','new_type','new_paint_color','new_manufacturer'])
df

,region,year,manufacturer,model,cylinders,fuel,odometer,description,title_status,transmission,drive,type,paint_color,state,price
0,auburn,2010.0,chevrolet,corvette grand sport,8 cylinders,gas,32742.0,Carvana is the safer way to buy a car During t...,clean,other,rwd,other,"custom, custom, red, red, gray, black, red, si...",al,35990
1,auburn,2014.0,hyundai,sonata,4 cylinders,gas,93600.0,I'll move to another city and try to sell my c...,clean,automatic,fwd,sedan,blue,al,7500
2,auburn,2006.0,bmw,x3 3.0i,6 cylinders,gas,87046.0,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,clean,automatic,,SUV,blue,al,4900
3,auburn,1974.0,chevrolet,c-10,4 cylinders,gas,190000.0,1974 chev. truck (LONG BED) NEW starter front ...,clean,automatic,rwd,pickup,blue,al,2000
4,auburn,2005.0,ford,f350 lariat,8 cylinders,diesel,116000.0,2005 Ford F350 Lariat (Bullet Proofed). This t...,lien,automatic,4wd,pickup,blue,al,19500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458208,wausau,2014.0,cadillac,srx,6 cylinders,gas,83790.0,2014 Cadillac SRX All Wheel Drive Luxury Colle...,clean,automatic,4wd,SUV,silver,wi,14995
458209,wausau,2018.0,hyundai,tucson,4 cylinders,gas,31888.0,2018 Hyundai Tucson SE 2.0 I4 AWD. Carfax ver...,clean,automatic,4wd,SUV,white,wi,17895
458210,wyoming,2014.0,cadillac,cts coupe awd,6 cylinders,gas,53000.0,"53,000 miles. Cosmetically, in near mint condi...",clean,automatic,4wd,coupe,black,wy,17900
458211,wyoming,2008.0,jeep,grand cherokee,8 cylinders,gas,164000.0,PRICE REDUCTION Turns out the engine is toast...,clean,automatic,4wd,SUV,white,wy,1300


In [41]:
#Drop all new columns
#Drop all columns worked on that have more than one entry

df = df.drop(columns=['new_description','newdrive','new_type','new_paint_color','new_manufacturer'])

new_df = df[df['paint_color'].str.split().str.len()<2]
new_df = new_df[new_df['type'].str.split().str.len()<2]
new_df = new_df[new_df['drive'].str.split().str.len()<2]
new_df = new_df[new_df['drive'].str.split().str.len()<3]

new_df = new_df. replace(r'^\s*$', np. nan, regex=True)

In [40]:
new_df

,region,year,manufacturer,model,cylinders,fuel,odometer,description,title_status,transmission,drive,type,paint_color,state,price
1,auburn,2014.0,hyundai,sonata,4 cylinders,gas,93600.0,I'll move to another city and try to sell my c...,clean,automatic,fwd,sedan,blue,al,7500
2,auburn,2006.0,bmw,x3 3.0i,6 cylinders,gas,87046.0,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,clean,automatic,,SUV,blue,al,4900
3,auburn,1974.0,chevrolet,c-10,4 cylinders,gas,190000.0,1974 chev. truck (LONG BED) NEW starter front ...,clean,automatic,rwd,pickup,blue,al,2000
4,auburn,2005.0,ford,f350 lariat,8 cylinders,diesel,116000.0,2005 Ford F350 Lariat (Bullet Proofed). This t...,lien,automatic,4wd,pickup,blue,al,19500
5,auburn,2016.0,toyota,tacoma double cab sr5,6 cylinders,gas,33290.0,Carvana is the safer way to buy a car During t...,clean,other,4wd,pickup,red,al,29590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458207,wausau,2017.0,audi,a4 premium plus quattro awd,NaN,gas,101270.0,2017 Audi A4 Premium Plus Quattro AWD ✔ We TA...,clean,automatic,,sedan,red,wi,19950
458208,wausau,2014.0,cadillac,srx,6 cylinders,gas,83790.0,2014 Cadillac SRX All Wheel Drive Luxury Colle...,clean,automatic,4wd,SUV,silver,wi,14995
458209,wausau,2018.0,hyundai,tucson,4 cylinders,gas,31888.0,2018 Hyundai Tucson SE 2.0 I4 AWD. Carfax ver...,clean,automatic,4wd,SUV,white,wi,17895
458210,wyoming,2014.0,cadillac,cts coupe awd,6 cylinders,gas,53000.0,"53,000 miles. Cosmetically, in near mint condi...",clean,automatic,4wd,coupe,black,wy,17900


In [42]:
new_df.isnull().sum()

region               0
year               921
manufacturer      7031
model             3642
cylinders       106566
fuel              2048
odometer         36176
description         70
title_status      1370
transmission      1300
drive            76546
type             44826
paint_color      34935
state                0
price                0
dtype: int64

In [48]:
finalDataset = new_df.dropna()
finalDataset

,region,year,manufacturer,model,cylinders,fuel,odometer,description,title_status,transmission,drive,type,paint_color,state,price
1,auburn,2014.0,hyundai,sonata,4 cylinders,gas,93600.0,I'll move to another city and try to sell my c...,clean,automatic,fwd,sedan,blue,al,7500
3,auburn,1974.0,chevrolet,c-10,4 cylinders,gas,190000.0,1974 chev. truck (LONG BED) NEW starter front ...,clean,automatic,rwd,pickup,blue,al,2000
4,auburn,2005.0,ford,f350 lariat,8 cylinders,diesel,116000.0,2005 Ford F350 Lariat (Bullet Proofed). This t...,lien,automatic,4wd,pickup,blue,al,19500
5,auburn,2016.0,toyota,tacoma double cab sr5,6 cylinders,gas,33290.0,Carvana is the safer way to buy a car During t...,clean,other,4wd,pickup,red,al,29590
6,auburn,2012.0,ford,mustang shelby gt500 coupe,8 cylinders,gas,9692.0,Carvana is the safer way to buy a car During t...,clean,other,rwd,coupe,blue,al,39990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458206,wausau,2013.0,chevrolet,captiva,4 cylinders,gas,140000.0,2013 Chevy Captiva - Brand new battery - Runs ...,clean,automatic,fwd,SUV,blue,wi,5300
458208,wausau,2014.0,cadillac,srx,6 cylinders,gas,83790.0,2014 Cadillac SRX All Wheel Drive Luxury Colle...,clean,automatic,4wd,SUV,silver,wi,14995
458209,wausau,2018.0,hyundai,tucson,4 cylinders,gas,31888.0,2018 Hyundai Tucson SE 2.0 I4 AWD. Carfax ver...,clean,automatic,4wd,SUV,white,wi,17895
458210,wyoming,2014.0,cadillac,cts coupe awd,6 cylinders,gas,53000.0,"53,000 miles. Cosmetically, in near mint condi...",clean,automatic,4wd,coupe,black,wy,17900


It is possible to get more rows by cleaning multiple entries instead of dropping them.